In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

# Define the CNN model
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = nn.functional.relu(self.conv1(x))
        out = nn.functional.max_pool2d(out, 2)
        out = nn.functional.relu(self.conv2(out))
        out = nn.functional.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = nn.functional.relu(self.fc1(out))
        out = self.fc2(out)
        return out

# Instantiate the model
model = PureCNN().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}")

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


100%|██████████| 9912422/9912422 [00:00<00:00, 303916017.87it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 94637260.80it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 139119154.26it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 5890701.54it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/10], Step [100/469], Loss: 0.2655
Epoch [1/10], Step [200/469], Loss: 0.1390
Epoch [1/10], Step [300/469], Loss: 0.0705
Epoch [1/10], Step [400/469], Loss: 0.0834
Epoch [2/10], Step [100/469], Loss: 0.0674
Epoch [2/10], Step [200/469], Loss: 0.0599
Epoch [2/10], Step [300/469], Loss: 0.0488
Epoch [2/10], Step [400/469], Loss: 0.1108
Epoch [3/10], Step [100/469], Loss: 0.0502
Epoch [3/10], Step [200/469], Loss: 0.0094
Epoch [3/10], Step [300/469], Loss: 0.0754
Epoch [3/10], Step [400/469], Loss: 0.0170
Epoch [4/10], Step [100/469], Loss: 0.0224
Epoch [4/10], Step [200/469], Loss: 0.0365
Epoch [4/10], Step [300/469], Loss: 0.0142
Epoch [4/10], Step [400/469], Loss: 0.0657
Epoch [5/10], Step [100/469], Loss: 0.0149
Epoch [5/10], Step [200/469], Loss: 0.0186
Epoch [5/10], Step [300/469], Loss: 0.0091
Epoch [5/10], Step [400/469], Loss: 0.0078
Epoch [6/10], Step [100/469], Loss: 0.0182
Epoch [6/10], Step [20

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(16, 3, activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(2),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10),
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
469/469 [==============================] - 5s 5ms/step - loss: 0.2703 - accuracy: 0.9219
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0700 - accuracy: 0.9784
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0510 - accuracy: 0.9837
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0399 - accuracy: 0.9878
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0323 - accuracy: 0.9903
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0266 - accuracy: 0.9915
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0223 - accuracy: 0.9932
Epoch 8/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0182 - accuracy: 0.9943
Epoch 9/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0171 - accuracy: 0.9943
Epoch 10/10
469/469 [==============